In [133]:
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from catboost import CatBoostClassifier

%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\PDudukin\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [134]:
data = pd.read_csv('tips.csv', encoding='cp1251')
data.head()

,UID,ID,Tipster,Date,Track,Horse,Bet Type,Odds,Result,TipsterActive
0,1,1.0,Tipster A,24/07/2015,Ascot,Fredricka,Win,8.00,Lose,True
1,2,2.0,Tipster A,24/07/2015,Thirsk,Spend A Penny,Win,4.50,Lose,True
2,3,3.0,Tipster A,24/07/2015,York,Straightothepoint,Win,7.00,Lose,True
3,4,4.0,Tipster A,24/07/2015,Newmarket,Miss Inga Sock,Win,5.00,Lose,True
4,5,5.0,Tipster A,25/07/2015,Ascot,Peril,Win,4.33,Win,True


In [135]:
data.shape

(38248, 10)

In [136]:
data.isnull().sum()

UID              0
ID               1
Tipster          1
Date             1
Track            1
Horse            1
Bet Type         1
Odds             1
Result           1
TipsterActive    1
dtype: int64

In [137]:
data[data['ID'].isnull()]

,UID,ID,Tipster,Date,Track,Horse,Bet Type,Odds,Result,TipsterActive
27804,"27805,81,Tipster Y,20/06/2015,Ascot,""First,fli...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
# Do the data correct
data.loc[27804,'UID'] = 27805
data.loc[27804,'ID'] = 81
data.loc[27804,'Tipster'] = 'Tipster Y'
data.loc[27804,'Date'] = '20/06/2015'
data.loc[27804,'Track'] = 'Ascot'
data.loc[27804,'Horse'] = 'First, flight'
data.loc[27804,'Bet Type'] = 'Win'
data.loc[27804,'Odds'] = 9
data.loc[27804,'Result'] = 'Lose'
data.loc[27804,'TipsterActive'] = False

In [139]:
data[data['ID'].isnull()]

,UID,ID,Tipster,Date,Track,Horse,Bet Type,Odds,Result,TipsterActive


In [140]:
data['Tipster'].unique()

array(['Tipster A', 'Tipster B', 'Tipster C', 'Tipster D', 'Tipster E',
       'Tipster F', 'Tipster G', 'Tipster H', 'Tipster I', 'Tipster J',
       'Tipster K', 'Tipster L', 'Tipster M', 'Tipster N', 'Tipster O',
       'Tipster P', 'Tipster Q', 'Tipster R', 'Tipster S', 'Tipster T',
       'Tipster U', 'Tipster V', 'Tipster W', 'Tipster X', 'Tipster Y',
       'Tipster Z', 'Tipster A1', 'Tipster B1', 'Tipster C1', 'Tipster D1',
       'Tipster E1'], dtype=object)

In [141]:
X = data[['Tipster','Date','Track','Horse','Bet Type','Odds','TipsterActive']]
y = data['Result']

dct = {'Win':1, 'Lose':0}
y = y.map(dct)

### CatBoost

In [161]:
def split(X, y, prc):
    
    border = np.floor((1-prc)*X.shape[0]).astype(int)
    print(border)
    
    X_train = X[:border]
    X_test = X[border:] 
    y_train = y[:border]
    y_test = y[border:]
    X_train.head()
    return X_train, X_test, y_train, y_test

In [162]:
X_train, X_test, y_train, y_test = split(X,y,0.3)

26773


In [170]:
y_test.values

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [182]:
# specify the training parameters 
model = CatBoostClassifier(iterations=500, depth=10, learning_rate=0.03, loss_function='Logloss', verbose=True)
#train the model
model.fit(X_train, y_train, cat_features=[0,1,2,3,4,6])
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
print("class = ", preds_class[:10])
print("proba = ", preds_proba[:10])

CatboostError: Invalid loss_fanction='Accuracy': for classifier use Logloss, CrossEntropy, MultiClass or custom objective object

In [180]:
def calculate_accuracy(data_pred,data_act):
    
    accuracy = 0.
    
    for i in range(len(data_pred)):       
        if data_pred[i] == data_act.values[i]:
            accuracy = accuracy + 1
        
    return accuracy/data_pred.shape[0]

In [181]:
calculate_accuracy(preds_class,y_test)

0.8415686274509804